In [1]:
# importing pandas
import pandas as pd

In [2]:
# importing and viewing the data
df = pd.read_csv('./data/apple_stock_data.csv')
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [3]:
# converting the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
# checking the conversion effectiveness
df['Date'].dtype

datetime64[ns, UTC]

In [6]:
# setting the 'Date' column as the dataframe index
df.set_index('Date', inplace=True)

In [8]:
#checking the dataframe indexes and their type
df.index

DatetimeIndex(['2023-11-02 00:00:00+00:00', '2023-11-03 00:00:00+00:00',
               '2023-11-06 00:00:00+00:00', '2023-11-07 00:00:00+00:00',
               '2023-11-08 00:00:00+00:00', '2023-11-09 00:00:00+00:00',
               '2023-11-10 00:00:00+00:00', '2023-11-13 00:00:00+00:00',
               '2023-11-14 00:00:00+00:00', '2023-11-15 00:00:00+00:00',
               ...
               '2024-10-21 00:00:00+00:00', '2024-10-22 00:00:00+00:00',
               '2024-10-23 00:00:00+00:00', '2024-10-24 00:00:00+00:00',
               '2024-10-25 00:00:00+00:00', '2024-10-28 00:00:00+00:00',
               '2024-10-29 00:00:00+00:00', '2024-10-30 00:00:00+00:00',
               '2024-10-31 00:00:00+00:00', '2024-11-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=252, freq=None)

In [9]:
# reducing the dataframe to the 'Close' column
df = df[['Close']]

In [10]:
# checking the new form of the dataframe
df.head()

,Close
Date,
2023-11-02 00:00:00+00:00,177.570007
2023-11-03 00:00:00+00:00,176.649994
2023-11-06 00:00:00+00:00,179.229996
2023-11-07 00:00:00+00:00,181.820007
2023-11-08 00:00:00+00:00,182.889999


## Choosing the Hybrid Models


    We will be using LSTM (Long Short-Term Memory) and Linear Regression models for this task. I chose LSTM because it effectively captures sequential dependencies and patterns in time-series data, which makes it suitable for modelling stock price movements influenced by historical trends.

    Linear Regression, on the other hand, is a straightforward model that captures simple linear relationships and long-term trends in data. By combining these two models into a hybrid approach, we leverage the LSTM’s ability to model complex time-dependent patterns alongside the Linear Regression’s ability to identify and follow broader trends. This combination aims to create a more balanced and accurate prediction system.

### Data normalization


In [11]:
# importing required library
from sklearn.preprocessing import MinMaxScaler

In [12]:
# declaring and initializing the scaler
scaler = MinMaxScaler(feature_range=(0,1))

In [13]:
# applying the scaler to the data
df['Close'] = scaler.fit_transform(df[['Close']])

In [14]:
# checking the result
df['Close'].head()

Date
2023-11-02 00:00:00+00:00    0.175853
2023-11-03 00:00:00+00:00    0.162983
2023-11-06 00:00:00+00:00    0.199077
2023-11-07 00:00:00+00:00    0.235311
2023-11-08 00:00:00+00:00    0.250280
Name: Close, dtype: float64

### Preparing data for LSTM

#### Creating sequences of a defined length(e.g: 60days)


In [16]:
# importing the required library
import numpy as np

In [19]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    
    return np.array(X), np.array(y)

In [20]:
seq_length = 60
X, y = create_sequences(df['Close'].values, seq_length)

In [21]:
X

array([[0.1758535 , 0.16298258, 0.19907662, ..., 0.3836038 , 0.37395072,
        0.32232785],
       [0.16298258, 0.19907662, 0.23531069, ..., 0.37395072, 0.32232785,
        0.27140452],
       [0.19907662, 0.23531069, 0.2502798 , ..., 0.32232785, 0.27140452,
        0.30581984],
       ...,
       [0.5907946 , 0.62702868, 0.67585339, ..., 0.92907118, 0.956911  ,
        0.96068834],
       [0.62702868, 0.67585339, 0.71684399, ..., 0.956911  , 0.96068834,
        0.9107444 ],
       [0.67585339, 0.71684399, 0.73489091, ..., 0.96068834, 0.9107444 ,
        0.85212657]])

In [22]:
y

array([0.27140452, 0.30581984, 0.29169009, 0.31729147, 0.3399553 ,
       0.3414942 , 0.32624523, 0.33365987, 0.30987682, 0.28035806,
       0.26790704, 0.26385005, 0.24216562, 0.23167317, 0.24230566,
       0.27098484, 0.2451036 , 0.22607729, 0.2466425 , 0.22971459,
       0.22034137, 0.2050924 , 0.14129836, 0.07162836, 0.05763844,
       0.05595971, 0.08016223, 0.10842194, 0.11513705, 0.08575833,
       0.11191942, 0.10660318, 0.12199219, 0.15500843, 0.19124229,
       0.08911577, 0.10184666, 0.08184116, 0.06589266, 0.11625627,
       0.09065467, 0.07036932, 0.05372127, 0.06505308, 0.05344163,
       0.0640739 , 0.04826521, 0.06533294, 0.03889198, 0.14045878,
       0.16158371, 0.10758258, 0.06127595, 0.04196978, 0.02853936,
       0.        , 0.01175149, 0.02658078, 0.05623957, 0.06841074,
       0.06015673, 0.11891439, 0.07456634, 0.06015673, 0.11233911,
       0.25713495, 0.23377179, 0.24342466, 0.2481814 , 0.273783  ,
       0.25251824, 0.29770565, 0.31379398, 0.34583104, 0.34750

In [23]:
#### Splitting the sequences into training and test sets(80-20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#### Building a sequential LSTM model with layers to capture the temporal dependencies in the data

In [27]:
# importing required library
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import LSTM, Dense

In [28]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

In [29]:
# compiling the model 
lstm_model.compile(optimizer='adam',loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1932
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0275
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0267
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0216
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0195
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0141
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0122
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0118
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0116
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0121
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0105
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0104
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0107
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0087
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0089
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━